# Lekce 10

## Systémy na doporučování obsahu

Systémy na doporučování obsahu (*recommendation engine*) se snaží z dostupných možností vybrat tu nejzajímavější pro konkrétního uživatele. Systémy na doporučování obsahu jsou běžnou součástí řady služeb, například e-shopů, streamovacích služeb, sociálních sítí, pracovních portálů. Pomocí těchto systémů se poskytovatelé snaží udržet uživatele co nejdéle na jejich platformě (aby jim mohli zobrazit co nejvíce reklamy), nabídnout produkt, který si uživatel s největší pravděpodobností koupí atd.

Systémy na doporučování obsahu mohou fungovat na dvou principech.

Filtrování na základě obsahu (*content based filtering*) využívá data o jednotlivých položkách (např. filmech, knihách atd.). Jestliže se uživateli (uživatelce) líbí nějaký obsah, systém mu nabídne obsah, který je nejvíce podobný. Při vývoji tohoto systému tedy řešíme podobnost jednotlivých položek (např. na základě popisu).

![content-based.png](content-based.png)

K vývoji nám stačí data o obsahu, tento přístup je tedy možné použít pro začínající službu. V některých případech může ale vést k "očividným" výsledkům, například pokud se někomu líbí první díl Star Wars, je mu nabídnutý druhý díl atd.

Kolaborativní filtrování (*collaborative filtering*) využívá data z chování uživatelů. Systém nějakému uživateli (uživatelce) vybírá takový obsah, který se líbil jemu podobným uživatelům (uživatelkám). Při vývoji tohoto systému tedy řešíme podobnost preferencí uživatelů (uživatelek) dané služby.

Pro vývoj je potřeba mít k dispozici dostatek dat o chování uživatelů. To může být problém například u nově vytvořených služeb, nových příspěvků na sociálních sítích, nových nebo nepřihlášených uživatelů/uživatelek atd. Pokud ale data máme k dispozici, může tento přístup generovat poměrně zajímavé výsledky.

![collaborative-filtering](collaborative-filtering.png)

Data mohou být založená na tzv. explicitním nebo implicitním feedbacku (*explicit feedback*, *implicit feedback*). Příkladem explicitního feedbacku je použití tlačítka "To se mi líbí", napsání pozitivní recenze atd. Příkladem implicitní vazby je dokoukání (nebo naopak nedokokání) filmu do konce, opětovné zhlédnutí atd.

### Filtrování na základě obsahu

V této části si vyzkoušíme filtrování na základě popisu. Využijeme dataset s 5000 filmy ze serveru [imdb.com](https://imdb.com). Pozor, jedná se o jiný dataset, než jaký jsme používali v minulé lekci. Dataset je uložený v souboru [tmdb_5000_movies.csv](tmdb_5000_movies.csv). Budeme využívat sloupec `overview`, kde jsou uložené slovní popisy filmů. Filmy, které nemají slovní popis, z dat vyřadíme.

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv("tmdb_5000_movies.csv")
data = data.dropna(subset=["overview"])
data.head()

,title,budget,gen_Action,gen_Adventure,gen_Animation,gen_Comedy,gen_Crime,gen_Documentary,gen_Drama,gen_Family,...,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,vote_average,vote_count
0,Avatar,237000000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,7.2,11800
1,Pirates of the Caribbean: At World's End,300000000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",6.9,4500
2,Spectre,245000000,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,6.3,4466
3,The Dark Knight Rises,250000000,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,7.6,9106
4,John Carter,260000000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",6.1,2124


Podíváme se, kolik nám zbylo filmů.

In [4]:
data.shape

(4800, 40)

Na sloupec `overview` opět použijeme algoritmus TF-IDF, který jsme již používali v minulé lekci. Tento algoritmus převede jednotlivé popisy (dokumenty) na číselnou matici. Každé slovo (nebo dvojice slov) budou mít svůj sloupeček a číslo v něm bude reflektovat přítomnost slova v dokumentu a přítomnost slova ve všech dokumentech. Nově přidáme dva parametry.

- `min_df` je minimální počet výskytů slova v dokumentech, aby nebylo vyřazeno. Potřebujeme slova, která se nachází alespoň ve dvou dokumentech, unikátní slovo nám nijak nepomůže propojit dokument s jiným dokumentem.
- `max_df` je maximální počet výskytů slova (zadáváme jako desetinné číslo, tj. 70 %). Slova, která jsou ve většině dokumentů, nám taky nepomůžou vybrat vhodný obsah.

Pro oba parametry platí následující pravidla:

- Pokud zadáme hodnotu v intervalu 0 až 1, je hodnota braná jako procentuální výskyt.
- Pokud zadáme hodnotu vyšší než 1, je hodnota braná jako výskyt v absolutní hodnotě.

Parametry `stop_words` a `ngram_range` jsme si již ukázali minule a jich význam je stále stejný. Dále použijeme metodu `fit_transform()` a na výsledná data se podíváme.

In [5]:
X = data["overview"]

vec = TfidfVectorizer(min_df=2, max_df=0.7, stop_words="english", ngram_range=(1, 2))
data_tfidf = vec.fit_transform(X)
# Převod na pole
data_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Jednotlivá slova, kterým čísla patří, získáme pomocí metody `get_feature_names_out()`.

In [6]:
vec.get_feature_names_out()

array(['00', '00 agent', '000', ..., 'zookeeper', 'zoologists', 'zorro'],
      dtype=object)

Abychom se na data mohli podívat v lepším formátu, můžeme si z nich opět sestavit pandas tabulku. Při vytváření tabulky řekneme, že popisky sloupců je možné získat pomocí metody `get_feature_names_out()`. Protože počet ani pořadí řádků se nijak nezměnily, načteme index tabulky ze sloupce `title` z původní tabulky s daty.

In [7]:
data_tfidf = pd.DataFrame(data_tfidf.toarray(), columns=vec.get_feature_names_out(), index=data["title"])
data_tfidf.head()

,00,00 agent,000,000 000,000 feet,000 foot,000 years,007,10,10 million,...,zombie outbreak,zombies,zone,zoo,zoo need,zooey,zooey deschanel,zookeeper,zoologists,zorro
title,,,,,,,,,,,,,,,,,,,,,
Avatar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pirates of the Caribbean: At World's End,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Spectre,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Dark Knight Rises,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
John Carter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Nyní si vybereme jeden film, například film [The Fugitive (Uprchlík)](https://www.imdb.com/title/tt0106977/) z roku 1993 s Harrísonem Fordem v hlavní roli. Seřadíme si hodnoty od nejvyšší po nejmenší. Vidíme, že především slovo "Richard" může být celkem matoucí, protože jde o jméno hlavní postavy filmu, ale k ději nemá žádnou vazbu. Naopak slova jako "killer", "wrongfully" nebo "pursuing team" můžou být poměrně důležitá a můžou nám pomoci najít dějově podobný film. (Film Uprchlík je o chirurgovi Richardovi Kimblovi, který byl nespravedlivě odsouzen za vraždu své ženy. Při převozu do vězení se mu podaří uprchnout. Poté hledá skutečného vraha a je při tom pronásledován skupinou policitů.)

In [8]:
data_tfidf.loc["The Fugitive"].sort_values(ascending=False).head(20)

richard              0.399296
killer               0.216403
discovers secrets    0.186469
wrongfully           0.186469
kimble               0.186469
death struggles      0.186469
pursuing team        0.186469
struggles expose     0.186469
team                 0.185769
gerard               0.180066
marshals             0.180066
wife                 0.177323
accused murdering    0.171041
samuel               0.167610
intricate            0.167610
chases               0.164638
deputy               0.162017
pursuing             0.159672
murdering            0.153832
expose               0.149211
Name: The Fugitive, dtype: float64

Abychom dokázali změřit podobnost mezi jednotlivými řádky tabulky `data_tfidf`, použijeme kosinovou podobnost.

#### Kosinová vzdálenost a podobnost

Kosinová vzdálenost (*cosine distance*) a kosinová podobnost (*cosine similarity*) jsou způsoby měření vzdálenosti, se kterým jsme se zatím nesetkali. Ukažme si, jak kosinová vzdálenost a podobnost fungují v dvourozměrném prostoru, tj. jak by změřila vzdálenost dvou hodnot z tabulky se dvěma sloupci. Body vidíme na obrázku níže. Kosinová podobnost je vypočítaná jako kosinus úhlu $\theta$, tj. úhlu, který svírají vektory, které směrují do obou bodů. Pokud bychom měli dva naprosto totožné body, úhel mezi jejich vektory by měl velikost 0 stupňů. Kosinová vzdálenost těchto bodů by byla 0, kosinová podobnost by byla 1.

![cosine.png](cosine.png)

Kosinovou vzdálenost a podobnost je možné spočítat i ve vícerozměrném prostoru a i v něm platí, že čím jsou si vektory podobnější, tím větší je hodnota kosinové podobnosti a menší hodnota kosinové vzdálenosti. My budeme využívat kosinovou podobnost. Kosinová podobnost je v modulu `scikit-learn` implementovaná ve funkci `cosine_similarity()`.

Funkci můžeme dát dvě hodnoty (v našem případě dva filmy) a ona spočítá kosinovou podobnost, tj. podobnost jejich popisů. Pokud funkci dáváme dva filmy, můžeme je vybrat s využitím indexu (a vlastnosti `loc`). Funkce ale očekává dvourozměrné pole.

Jednorozměrné pole můžeme ze série získat s využitím vlastnosti `.values`.

In [9]:
data_tfidf.loc["The Fugitive"].values

array([0., 0., 0., ..., 0., 0., 0.])

Abychom získali dvourozměrnou matici, můžeme použít metodu `reshape()`. Té dáme parametry `1` a `-1`, čímž získáme dvourozměrné pole.

Hodnota `1` znamená, že chceme mít pole, které má jeden řádek. A hodnota `-1` udělá pole dvourozměrné.

In [10]:
data_tfidf.loc["The Fugitive"].values.reshape(1, -1)

array([[0., 0., 0., ..., 0., 0., 0.]])

Spočítejme nyní podobnost například mezi filmem The Fugitive a Witness

In [11]:
cosine_similarity(data_tfidf.loc["The Fugitive"].values.reshape(1, -1), data_tfidf.loc["Witness"].values.reshape(1, -1))

array([[0.01564247]])

Podobnost je sice poměrně nízká (hodnota je blízká 0), absolutní hodnota nemusí být příliš důležitá. Zkusme ještě spočítat podobnost mezi filmy The Fugitive a Titanic.

In [12]:
cosine_similarity(data_tfidf.loc["The Fugitive"].values.reshape(1, -1), data_tfidf.loc["Titanic"].values.reshape(1, -1))

array([[0.00693092]])

Vidíme, že pro tyto fimy je podobnost větší. Náš systém by tedy nabídl uživateli, kterému se líbil film The Fugitive, film Witness a nikoli film Titanic.

V celé datové sadě se ale může nacházet ještě vhodnější film. Abychom získali obecná data pro doporučení libovolného filmu, použijeme funkci  `cosine_similarity` na celou tabulku `cosine_similarity()`. Tento příkaz spočítá kosinovou podobnost pro každou dvojici filmů. Získáme tedy čtvercovou matici, kde se počet řádků a počet sloupců rovná počtu filmů v původní tabulce.

In [13]:
cosine_similarity_array = cosine_similarity(data_tfidf)
cosine_similarity_array

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02190503, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01450648, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02190503, 0.01450648, ..., 1.        , 0.01549051,
        0.00614697],
       [0.        , 0.        , 0.        , ..., 0.01549051, 1.        ,
        0.01001404],
       [0.        , 0.        , 0.        , ..., 0.00614697, 0.01001404,
        1.        ]])

Pro snazší práci s daty opět provedeme převod na tabulku. Názvy sloupců i řádků (index) jsou dané jmény filmů.

In [14]:
cosine_similarity_df = pd.DataFrame(cosine_similarity_array, index=data_tfidf.index, columns=data_tfidf.index)
cosine_similarity_df.head()

title,Avatar,Pirates of the Caribbean: At World's End,Spectre,The Dark Knight Rises,John Carter,Spider-Man 3,Tangled,Avengers: Age of Ultron,Harry Potter and the Half-Blood Prince,Batman v Superman: Dawn of Justice,...,On The Downlow,Sanctuary: Quite a Conundrum,Bang,Primer,Cavite,El Mariachi,Newlyweds,"Signed, Sealed, Delivered",Shanghai Calling,My Date with Drew
title,,,,,,,,,,,,,,,,,,,,,
Avatar,1.000000,0.000000,0.0,0.020459,0.000000,0.024428,0.000000,0.033653,0.000000,0.000000,...,0.000000,0.0,0.027932,0.040753,0.000000,0.0,0.0,0.000000,0.000000,0.000000
Pirates of the Caribbean: At World's End,0.000000,1.000000,0.0,0.000000,0.034509,0.000000,0.000000,0.021569,0.000000,0.000000,...,0.000000,0.0,0.007011,0.000000,0.000000,0.0,0.0,0.021905,0.000000,0.000000
Spectre,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.028307,0.019265,0.000000,...,0.025571,0.0,0.000000,0.000000,0.018272,0.0,0.0,0.014506,0.000000,0.000000
The Dark Knight Rises,0.020459,0.000000,0.0,1.000000,0.009002,0.003669,0.011148,0.021395,0.013906,0.136604,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.028649,0.035299,0.028564
John Carter,0.000000,0.034509,0.0,0.009002,1.000000,0.000000,0.009827,0.035313,0.000000,0.017015,...,0.012130,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.006163,0.000000,0.000000


Matice je symetrická. Pokud nás zajímají podobnosti jednotlivých filmů s filmem The Fugitive, můžeme vyjmut řádek (pomocí `.loc`) nebo sloupec (s využitím hranatých závorek). Jednodušší bude práce se sloupcem, protože budeme chtít data seřadit.

In [15]:
distances_the_fugitive = cosine_similarity_df["The Fugitive"]
distances_the_fugitive

title
Avatar                                      0.000000
Pirates of the Caribbean: At World's End    0.000000
Spectre                                     0.000000
The Dark Knight Rises                       0.018202
John Carter                                 0.000000
                                              ...   
El Mariachi                                 0.048717
Newlyweds                                   0.000000
Signed, Sealed, Delivered                   0.032423
Shanghai Calling                            0.000000
My Date with Drew                           0.000000
Name: The Fugitive, Length: 4800, dtype: float64

Zobrazíme si tedy 10 nejvíce podobných filmů. Nejvíce "podobný" je film "The Rocket: The Legend of Rocket Richard", který ale dějově nemá s filmem The Fugitive nic společného. Podobnost je daná čistě tím, že hlavní postavy obou filmů se jmenují Richard. To bychom mohli vyřešit například tím, že bychom zohlednili zánr filmu. Naopak následující dva filmy The Iceman a The Life of David Gale lze považovat za dobré tipy.

In [16]:
distances_the_fugitive.sort_values(ascending=False).head(10)

title
The Fugitive                                1.000000
The Rocket: The Legend of Rocket Richard    0.207967
The Iceman                                  0.171206
The Life of David Gale                      0.165951
Married Life                                0.157624
The Work and The Story                      0.153640
A Time to Kill                              0.112665
Austin Powers in Goldmember                 0.109479
Somewhere in Time                           0.108948
High Anxiety                                0.100456
Name: The Fugitive, dtype: float64

### Čtení na doma: Výběr na základě uživatelského profilu

Pokud uživatel používá nějakou službu delší dobu, budeme určitě vědět o více produktech (např. filmech), které se mu líbily. V takovém případě nebudeme hledat podobnost mezi dvěma produkty, ale mezi uživatelským profilem a produkty, které budou pro uživatele nové (např. filmy, které ještě neviděl). Uvažujme například uživatele (uživatelku), které se líbily firmy Notting Hill, Titanic, The Great Gatsby a The Lovers. To, že se filmy líbily, můžeme poznat podle explicitního (uživatel použil tlačítko "To se mi líbí") nebo implicitního (uživatel zhlédl celý film) feedbacku.

Vytvoříme tedy proměnnou `data_user_profile`, která bude reprezentovat preference uživatele (uživatelky). Ty získáme tak, že spočítáme průměrné hodnoty jednotlivých slov (a dvojic slov) pro výše uvedené filmy. Abychom měli data ve dvourozměrné matici, oipět je třeba použít metodu `reshape()`.

In [17]:
favorite_movies = ["Notting Hill", "Titanic", "The Great Gatsby", "The Lovers"]
data_user_profile = data_tfidf.loc[favorite_movies]
data_user_profile = data_user_profile.mean()
data_user_profile = data_user_profile.values.reshape(1, -1)
data_user_profile

array([[0., 0., 0., ..., 0., 0., 0.]])

Nyní zkusme najít filmy, které budou nejvíce podobného uživatelskému profilu. Nejprve z původní tabulky vyřadíme filmy ze seznamu `favorite_movies`. Poté spočítáme kosinové vzdálenosti mezi uživatelským profilem a jednotlivými filmy. Vznikne matice s jedním sloupcem, který pro každý film udává kosinovou vzdálenost mezi popisem filmu a uživatelským profilem.

In [18]:
data_tfidf_not_seen = data_tfidf.drop(favorite_movies, axis=0)
cosine_similarity_array_not_seen = cosine_similarity(data_tfidf_not_seen, data_user_profile)
cosine_similarity_array_not_seen

array([[0.02648553],
       [0.02318485],
       [0.        ],
       ...,
       [0.01260339],
       [0.00843268],
       [0.02732532]])

Data opět převedeme na tabulku, abychom viděli názvy filmů, které náš systém danému uživateli doporučí. Jako indexy použijeme názvy filmů z tabulky `data_tfidf_not_seen`. Nakonec data seřadíme. Vidíme, že náš systém by doporučil filmy Romance & Cigarettes, Four Weddings and a Funeral a Rent.

In [19]:
user_recommendation = pd.DataFrame(cosine_similarity_array_not_seen, index=data_tfidf_not_seen.index, columns=["score"])
user_recommendation.sort_values(by="score", ascending=False).head()

,score
title,
Romance & Cigarettes,0.130770
Four Weddings and a Funeral,0.128872
Rent,0.121126
The Little Prince,0.112538
Dil Jo Bhi Kahey...,0.100109


### Čtení na doma: Kolaborativní filtrování

U kolaborativního filtrování se řídíme preferencemi jednotlivých uživatelů a uživatelek. Využijeme data ze seouboru [user_ratings.csv](user_ratings.csv), který obsahuje uživatelská hodnocení fimů. U hodnocení víme ID uživatele (`userId`), bodové hodnocení filmu (sloupec `rating`) a název filmu (sloupec `title`).

In [20]:
data_ratings = pd.read_csv("user_ratings.csv")
data_ratings = data_ratings[["userId", "rating", "title"]].drop_duplicates(subset=["userId", "title"], keep="last")
data_ratings.head()

,userId,rating,title
0,1,4.0,Toy Story (1995)
1,5,4.0,Toy Story (1995)
2,7,4.5,Toy Story (1995)
3,15,2.5,Toy Story (1995)
4,17,4.5,Toy Story (1995)


Pro další práci potřebujeme mít trochu jinou strukturu tabulky. Měli bychom mít názvy filmů jako sloupce, takže jeden řádek tabulky bude představovat všechna hodnocení jednoho uživatele (uživatelky). K tomu můžeme využít funkce `pivot()`. Pozor, nejedná se o funkci `pivot_table()`! Rozdíl mezi `pivot()` a `pivot_table()` je v tom, že `pivot()` neprovádí žádnou agregaci, ale pouze "přeskládá" hodnoty z jednoho sloupce do více sloupců. Z toho důvodu zadáváme funkci `pivot` pouze tři parametry: `index` (názvy řádků - index), `columns` (názvy sloupců) a `values` (hodnoty). Nezadáváme agregační funkci.

In [21]:
data_ratings_pivot = data_ratings.pivot(index="userId", columns="title", values="rating")
data_ratings_pivot

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN


Podívejme se nyní na hodnocení uživatele (nebo uživatelky) s id 2. Jeho hodnocení zjistíme jako řádek s id 2. Pozor na to, že sloupce v původní tabulce jsou čísla, musíme tedy do `loc` vložít hodnotu 2 jako číslo, tj. bez uvozovek. Pokud bychom zadali číslo s uvozovkami, modul `pandas` by hledal řetězec 2 a nic by nenašel, protože řetězec 2 je jiná hodnota než číslo 2.

Vidíme, že máme celkem 29 hodnocení a jsou zastoupena jak vysoká, tak nízká hodnocení.

In [22]:
data_ratings_pivot.loc[2].dropna().sort_values()

title
The Drop (2014)                                         2.0
Girl with the Dragon Tattoo, The (2011)                 2.5
Zombieland (2009)                                       3.0
Shawshank Redemption, The (1994)                        3.0
Interstellar (2014)                                     3.0
Exit Through the Gift Shop (2010)                       3.0
Collateral (2004)                                       3.5
Django Unchained (2012)                                 3.5
Dark Knight Rises, The (2012)                           3.5
Ex Machina (2015)                                       3.5
Whiplash (2014)                                         4.0
Tommy Boy (1995)                                        4.0
Talladega Nights: The Ballad of Ricky Bobby (2006)      4.0
Shutter Island (2010)                                   4.0
Louis C.K.: Hilarious (2010)                            4.0
Kill Bill: Vol. 1 (2003)                                4.0
Departed, The (2006)              

Velkým problémem dat tohoto typu je, že v nich je obrovské množství prázdných hodnot. Pro taková data se používá označení řídká (*sparse*). My si ukážeme velmi jednoduchý přístup k řešení tohoto problému. Nemůžeme nyní ale jen nahradit chybějící hodnoty 0, protože tím bychom zkresili průměrná hodnocení filmu. Využijeme následující postup.

1. Pro každý film spočteme průměrné hodnocení.
1. Od každého hodnocení odečteme průměrné hodnocení.
1. Nyní můžeme prázdné hodnoty nahradit 0, aniž bychom ovlivnili průměrné hodnocení filmů.

In [23]:
avg_ratings = data_ratings_pivot.mean()
avg_ratings

title
'71 (2014)                                   4.000000
'Hellboy': The Seeds of Creation (2004)      4.000000
'Round Midnight (1986)                       3.500000
'Salem's Lot (2004)                          5.000000
'Til There Was You (1997)                    4.000000
                                               ...   
eXistenZ (1999)                              3.863636
xXx (2002)                                   2.770833
xXx: State of the Union (2005)               2.000000
¡Three Amigos! (1986)                        3.134615
À nous la liberté (Freedom for Us) (1931)    1.000000
Length: 9719, dtype: float64

In [24]:
data_ratings_pivot_standardized = data_ratings_pivot.sub(avg_ratings, axis=1)
data_ratings_pivot_standardized = data_ratings_pivot_standardized.fillna(0)
data_ratings_pivot_standardized

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.865385,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0000,0.000000,0.0,0.0,0.636364,0.729167,0.0,0.000000,0.0


#### Dopočítání hodnot

Podívejme se třeba na hodnocení filmu Pulp Fiction od uživatele s id 1 a na něm si ukážeme, jak vznikly hodnoty v tabulce `data_ratings_pivot_standardized`. Film dostal od uživatele hodnocení 3 body. Nyní vidíme, že má hodnocení -1.197068. Kde se toto číslo vzalo? Rozeberme si postup výpočtu.

In [25]:
data_ratings_pivot.loc[1, "Pulp Fiction (1994)"]

3.0

V tabulce `data_ratings_pivot_standardized` je hodnocení filmu Pulp Fiction -1.197068403908795.

In [26]:
data_ratings_pivot_standardized.loc[1, "Pulp Fiction (1994)"]

-1.197068403908795

Podívejme se na průměrné hodnocení tohoto filmu v původních tabulce `avg_ratings`. To má hodnotu 4.197068403908795.

In [27]:
avg_ratings.loc["Pulp Fiction (1994)"]

4.197068403908795

K číslu se dopočítáme, pokud vezmeme hodnocení 3 a odečteme od něj průměrné hodnocení filmu. Tím získáme hodnotu -1.197068403908795, což je stejná hodnota, jaká byla v 

In [28]:
3 - 4.197068403908795

-1.197068403908795

#### Podobnost uživatelů

Naším cílem nyní bude vyhledat k našemu uživateli č. 1 uživatel a uživatelky, kteří jsou mu (jí) nejvíce podobní. K výpočtu podobnosti opět můžeme využít kosinovou vzdálenost. Jednou z jejích hodnot je, že jí nevadí obrovské množství chybějících hodnot, jimi ovlivněná není. Pro výpočet podobnosti uživatelů využijeme tabulku se standardizovanými hodnoceními, tj. tabulku `data_ratings_pivot_standardized`. Výsledná data opět převedeme na tabulku.

In [29]:
similarities = cosine_similarity(data_ratings_pivot_standardized)
similarities = pd.DataFrame(similarities, index=data_ratings_pivot_standardized.index, columns=data_ratings_pivot_standardized.index)
similarities.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.005147,-0.045052,0.005082,0.018204,0.009896,-0.011553,0.027520,0.029854,-0.019043,...,0.030035,0.000167,-0.078620,-0.016158,-0.064228,-0.017452,0.036311,-0.092671,-0.005583,0.052643
2,0.005147,1.000000,0.000000,-0.018459,0.088285,-0.009396,0.008564,-0.035152,0.000000,-0.002004,...,-0.062581,-0.019606,-0.003878,0.000000,0.000000,0.022969,-0.015052,0.000654,0.049596,0.051937
3,-0.045052,0.000000,1.000000,-0.013393,-0.034685,0.015053,0.000000,-0.034564,0.000000,0.000000,...,-0.064929,-0.013092,0.026124,0.000000,0.001096,-0.053438,-0.000739,0.007190,0.000000,0.019865
4,0.005082,-0.018459,-0.013393,1.000000,-0.059430,0.013483,0.070708,0.025238,-0.009606,0.102568,...,-0.082818,0.057897,-0.015017,-0.005770,0.035938,-0.020737,0.011569,-0.029545,0.006883,-0.034780
5,0.018204,0.088285,-0.034685,-0.059430,1.000000,-0.001382,0.003460,-0.128999,0.000000,-0.029349,...,-0.010424,-0.064583,0.014181,-0.029109,-0.057051,0.010026,-0.001006,-0.024182,0.118276,-0.012240


Zobrazíme si, kteří uživatelé do 1 nejvíce podobných uživatelů patří. Nemáme žádný klíč k tomu, kolik uživatelů vybrat, zobrazíme tady 10 nejbližších. Do nich se počítá i sám uživatel 1, protože pro něj je kosinová podobnost 1 (tj. nejvyšší možná podobnost).

In [30]:
n_users = 10
user_1_similarities = similarities.loc[2]
user_1_similarities = user_1_similarities.sort_values(ascending=False)
user_1_similarities.head(n_users)

userId
2      1.000000
189    0.174249
145    0.170194
524    0.120279
468    0.104534
564    0.103556
379    0.093442
5      0.088285
329    0.087958
378    0.086594
Name: 2, dtype: float64

Pomocí `iloc[]` nyní vyberu `n_users` nejvíce podobných uživatelů, samotného uživatele 1 už vyřadíme.

In [31]:
similar_users = user_1_similarities.iloc[1:n_users+1].index
similar_users


Index([189, 145, 524, 468, 564, 379, 5, 329, 378, 272], dtype='int64', name='userId')

V tabulce níže máme hodnocení filmů od 10 vybraných uživatelů. Aby bylo hodnocení relevantní, vybereme pouze filmy, které byly ohodnoceny alespoň od 3 těchto uživatelů. K tomu využijeme metodu `dropna()`, kterou již známe, ale tentokrát ji použijeme poměrně nezvykle.

- Jako hodnotu parametru `axis` použijeme `1`, tj. chceme, aby metoda odebírala sloupečky, nikoli řádky. My totiž z výběru chceme vyřadit filmy, které mají málo hodnocení, a filmy jsou v tabulce jako sloupce.
- Nechceme ale ponechat pouze filmy hodnocené od všech vybraných uživatelů, to by nám v tabulce moc filmů nezbylo. Proto použijeme parametr `tresh=3` (zkratka od anglického výrazu Treshold), který ponechá sloupečky, které mají alespoň tři hodnoty.

In [32]:
data_ratings_pivot_similar = data_ratings_pivot.loc[similar_users]
data_ratings_pivot_similar = data_ratings_pivot_similar.dropna(axis=1, thresh=3)
data_ratings_pivot_similar

title,Ace Ventura: Pet Detective (1994),Aladdin (1992),Apollo 13 (1995),Batman (1989),Batman Forever (1995),Beauty and the Beast (1991),Braveheart (1995),Clear and Present Danger (1994),Cliffhanger (1993),Clueless (1995),...,Seven (a.k.a. Se7en) (1995),"Shawshank Redemption, The (1994)","Silence of the Lambs, The (1991)",Stargate (1994),Terminator 2: Judgment Day (1991),Toy Story (1995),True Lies (1994),"Truman Show, The (1998)","Usual Suspects, The (1995)",Waterworld (1995)
userId,,,,,,,,,,,,,,,,,,,,,
189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,2.0,4.0,3.0,3.0,3.0,4.0,NaN,NaN,3.0,NaN,...,NaN,3.0,NaN,2.0,NaN,5.0,3.0,NaN,5.0,3.0
524,3.0,4.0,5.0,3.0,3.0,NaN,3.0,5.0,NaN,NaN,...,5.0,1.0,4.0,3.0,5.0,4.0,4.0,NaN,2.0,1.0
468,3.0,3.0,5.0,3.0,3.0,5.0,4.0,2.0,3.0,5.0,...,5.0,3.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,3.0
564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
379,2.0,NaN,5.0,3.0,3.0,NaN,5.0,4.0,3.0,NaN,...,3.0,3.0,NaN,3.0,4.0,NaN,4.0,NaN,NaN,3.0
5,3.0,4.0,3.0,3.0,3.0,5.0,4.0,3.0,NaN,3.0,...,NaN,3.0,NaN,2.0,3.0,4.0,2.0,NaN,4.0,NaN
329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN
378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.5,4.5,NaN,NaN,4.5,NaN,4.0,NaN,NaN


Nakonec spočítáme průměrná hodnocení pro jednotlivé filmy a seřadíme je. U těchto filmů tedy můžeme předpokládat větší šanci, že by se mohly líbit i uživateli 2.

In [33]:
average_ratins_by_similar_users = data_ratings_pivot_similar.mean().sort_values(ascending=False)
average_ratins_by_similar_users.head(20)

title
Dances with Wolves (1990)                                    4.750000
Beauty and the Beast (1991)                                  4.666667
Fugitive, The (1993)                                         4.666667
Seven (a.k.a. Se7en) (1995)                                  4.333333
Toy Story (1995)                                             4.300000
Schindler's List (1993)                                      4.250000
Apollo 13 (1995)                                             4.200000
Forrest Gump (1994)                                          4.000000
Braveheart (1995)                                            4.000000
Terminator 2: Judgment Day (1991)                            3.875000
Clueless (1995)                                              3.833333
Crimson Tide (1995)                                          3.750000
Aladdin (1992)                                               3.750000
Mask, The (1994)                                             3.666667
Outbreak (1995

#### Čtení na doma: Odebrání filmů, které uživatel již viděl

Většině uživatelů asi nemá smysl doporučovat filmy, které už viděli, protože dají přednost něčemu novému. Můžeme tedy z našeho seznamu odebrat filmy, které už uživatel 2 ohodnotil. Nejprve si zjistíme, které to jsou. Pokud vybereme z tabulky hodnocení uživatele 2 a odebereme prázdné hodnoty, pak indexy vybrané série budou představovat filmy, které uživatel již viděl. Výsledek si uložíme do proměnné `seen_by_user_2`.

In [34]:
seen_by_user_2 = data_ratings_pivot.loc[2].dropna().index
seen_by_user_2

Index(['Collateral (2004)', 'Dark Knight Rises, The (2012)',
       'Dark Knight, The (2008)', 'Departed, The (2006)',
       'Django Unchained (2012)', 'Ex Machina (2015)',
       'Exit Through the Gift Shop (2010)',
       'Girl with the Dragon Tattoo, The (2011)', 'Gladiator (2000)',
       'Good Will Hunting (1997)', 'Inception (2010)',
       'Inglourious Basterds (2009)', 'Inside Job (2010)',
       'Interstellar (2014)', 'Kill Bill: Vol. 1 (2003)',
       'Louis C.K.: Hilarious (2010)', 'Mad Max: Fury Road (2015)',
       'Shawshank Redemption, The (1994)', 'Shutter Island (2010)',
       'Step Brothers (2008)',
       'Talladega Nights: The Ballad of Ricky Bobby (2006)', 'The Drop (2014)',
       'The Jinx: The Life and Deaths of Robert Durst (2015)',
       'Tommy Boy (1995)', 'Town, The (2010)', 'Warrior (2011)',
       'Whiplash (2014)', 'Wolf of Wall Street, The (2013)',
       'Zombieland (2009)'],
      dtype='object', name='title')

Nakonec tyto filmy odebereme z filmů, které chceme doporučit. Pomocí dotazu vybereme ze série `average_ratins_by_similar_users` položky, jejichž index (i u této série jsou index názvy filmů) není v  `seen_by_user_2`

In [35]:
average_ratins_by_similar_users = average_ratins_by_similar_users[~average_ratins_by_similar_users.index.isin(seen_by_user_2)]
average_ratins_by_similar_users.head(20)

title
Dances with Wolves (1990)                                    4.750000
Beauty and the Beast (1991)                                  4.666667
Fugitive, The (1993)                                         4.666667
Seven (a.k.a. Se7en) (1995)                                  4.333333
Toy Story (1995)                                             4.300000
Schindler's List (1993)                                      4.250000
Apollo 13 (1995)                                             4.200000
Forrest Gump (1994)                                          4.000000
Braveheart (1995)                                            4.000000
Terminator 2: Judgment Day (1991)                            3.875000
Clueless (1995)                                              3.833333
Crimson Tide (1995)                                          3.750000
Aladdin (1992)                                               3.750000
Mask, The (1994)                                             3.666667
Outbreak (1995

## Zdroje a odkazy

- [Understanding Cosine Similarity in Python with Scikit-Learn](https://memgraph.com/blog/cosine-similarity-python-scikit-learn)
- [What is the 'More Like This' section?](https://help.imdb.com/article/imdb/discover-watch/what-is-the-more-like-this-section/GPE7SPGZREKKY7YN?ref_=cons_tt_rec_lm#)
- [Beginner Tutorial: Recommender Systems in Python](https://www.datacamp.com/tutorial/recommender-systems-python)
- [Build a Recommendation Engine With Collaborative Filtering](https://realpython.com/build-recommendation-engine-collaborative-filtering/)
- [How Recommendation Systems Tackle the Cold Start Problem](https://www.yusp.com/blog-posts/cold-start-problem/)
- [Overcoming Echo Chambers in Recommendation Systems (Using Movie Ratings)](https://towardsdatascience.com/overcoming-echo-chambers-in-recommendation-systems-using-movie-ratings-3686f50fc053) (placená část Towards Data Science)
- [Introduction to recommender systems](https://towardsdatascience.com/introduction-to-recommender-systems-6c66cf15ada)
- [Recommender Systems — A Complete Guide to Machine Learning Models](https://towardsdatascience.com/recommender-systems-a-complete-guide-to-machine-learning-models-96d3f94ea748)

## Cvičení

Náš systém se orientoval čistě podle popisu děje filmu. Pokud nemáme k dispozici nic jiného, než slovní popis, je často nutné pracovat s ním. V našem datasetu ale máme ale další strukturovaná data, například žánr filmu. Ten může být užitečný. Vraťme se třeba k filmu The Fugitive. Uživatel nebo uživatelka, kterému (které) se tento film líbil, může preferovat filmy s podobnými žánry. Hypotetická romantická komedie o uprchlém vězni, který se na útěku zamiluje, může být méně lákavá než jiný akční thriller s detektivní zápletkou.

Žánry filmu máme v datech uvedeny ve sloupečcích, které začínaji `gen_`. 

Načti si nejprve znovu data a ponech si v nich pouze sloupce se žánry pomocí kódu níže.

In [36]:
data = pd.read_csv("tmdb_5000_movies.csv")
# Nastavíme index
data = data.set_index("title")
# Ponecháme v datech pouze sloupce od gen_Action po gen_Western
data_genres = data.loc[:, "gen_Action":"gen_Western"]
data_genres.head()

,gen_Action,gen_Adventure,gen_Animation,gen_Comedy,gen_Crime,gen_Documentary,gen_Drama,gen_Family,gen_Fantasy,gen_Foreign,gen_History,gen_Horror,gen_Music,gen_Mystery,gen_Romance,gen_Science Fiction,gen_TV Movie,gen_Thriller,gen_War,gen_Western
title,,,,,,,,,,,,,,,,,,,,
Avatar,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Pirates of the Caribbean: At World's End,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Spectre,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Dark Knight Rises,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
John Carter,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Podívej se na žánry filmu The Fugitive s využitím `.loc`.

Žárny filmů vyzkoušíme porovnat nikoli pomocí cosinové vzdálenosti, ale vyzkoušíme Jaccardovovo skóre. Jaccardovo skóre je založeno na následujícím vzorečku: (počet žánrů, které mají filmy společné / počet žánrů, které se vyskytují alespoň u jednoho filmu).

Uvažuj třeba hypotetický film A, který má žánr Comedy a Romance, a film B, který má žárn Comedy a History. Oba filmy mají společný jeden žánr (Comedy). Celkem se u obou filmů objevují žárny Comedy, Romance a History, tj. tři žánry. Jaccardovo skóre pro tyto dva filmy je 1/3.

Proveď import funkce `jaccard_score` pomocí příkazu níže.

In [37]:
from sklearn.metrics import jaccard_score

Nyní zkus spočítat Jaccardovo skóre pro filmy The Fugitive a Witness. Uděláš to tak, že funkci `jaccard_score` dáš dva parametry - sérii s žánry filmu The Fugitive a sérii s žánry filmu Witness. Sérii s žánry The Fugitive už jsi získala v předchozím kroku a sérii s žánry Witness získáš stejným způsobem.

Měla by ti vyjít hodnota přibližně 0.2857.

Dále zkus to samé pro film The Fugitive a Titanic. Mělo by ti vyjít přibližně 0.142856.

Nyní zkus vypočítat Jaccardovy vzdálenosti pro všechny filmy. Proveď následující importy:

In [38]:
from scipy.spatial.distance import pdist, squareform

Nejprve použij funkci `pdist`, která vypočítá Jaccardovy vzdálenosti pro všechny dvojice filmů. Funkci dej jako první parametr matici `data` a jako druhý parametr `metric="jaccard"`. Výsledek ulož do proměnné `jaccard_distances`. Výsledkem je jednorozměrné pole.

Jako druhý krok použij funkci `squareform`, která převede jednorozměrné pole na dvourozměrnou matici. Funkci volej pouze s jedním parametrem - proměnnou `jaccard_distances`. Výsledek opět ulož do proměnné s názvem `jaccard_distances`. Nakonec je je potřeba spustit tento řádek

```py
jaccard_distances = 1 - jaccard_distances
```

Funkce totiž počítá vzdálenosti, nikoli Jaccardovo skóre. Pro Jaccardovo skóre a Jaccardovu vzdálenost platí vztah: Jaccardovo skóre = 1 - Jaccardova vzdálenost.

Výsledek poté převeď na pandas tabulku stejným způsobem, jako jsme to dělali v lekci. Jako index (parametr `index`) a názvy sloupců (parametr `columns`) použij indexy z původní tabulky data (tj. `data.index`).

Poté vyber sloupec s filmem The Fugitive a seřaď hodnoty v něm od největší po nejmenší. Filmy nahoře jsou nyní nejvíce podobné a algortimus postavený na žánru filmu by je doporučil.

### Bonus

Podívej se na kód níže. Obsahuje metody a funkce, které jsme si ukazovali dříve, především v prvních třech lekcích. Dokážeš si vzpomenout, co tyto funkce dělají, a odhadnout výsledek? Pokud ne, podívej se na výstup níže.

In [39]:
data['release_date'] = pd.to_datetime(data['release_date'])
data['release_year'] = data['release_date'].dt.year

bins = [1979, 1989, 1999, 2009, 2019, 2029]
labels = ['1980s', '1990s', '2000s', '2010s', '2020s']

data['decade'] = pd.cut(data['release_year'], bins=bins, labels=labels)

Níže je nápověda, která zobrazuje nově přidaný sloupec `decade`.

In [41]:
data[['release_date', 'release_year', 'decade']].head()

,release_date,release_year,decade
title,,,
Avatar,2009-12-10,2009.0,2000s
Pirates of the Caribbean: At World's End,2007-05-19,2007.0,2000s
Spectre,2015-10-26,2015.0,2010s
The Dark Knight Rises,2012-07-16,2012.0,2010s
John Carter,2012-03-07,2012.0,2010s


Níže jsou komentáře k jednotlivým řádkům

In [ ]:
# Převod sloupce 'release_date' na datový typ datetime
data['release_date'] = pd.to_datetime(data['release_date'])

# Extrahování roku z 'release_date' a uložení do nového sloupce 'release_year'
data['release_year'] = data['release_date'].dt.year

# Definice rozmezí pro dekády
bins = [1979, 1989, 1999, 2009, 2019, 2029]

# Definice popisek pro jednotlivé dekády
labels = ['1980s', '1990s', '2000s', '2010s', '2020s']

# Kategorizace roku vydání filmu do dekád a přidání do nového sloupce 'decade'
data['decade'] = pd.cut(data['release_year'], bins=bins, labels=labels)

Dekádu jsme zjišťovali, protože každá dekáda měla z pohledu kinematografie jistá specifika a například v devadesátých letech se natáčely filmy jinak, než jak se natáčí dnes. A pokud se někomu líbí spíše starší filmy, pravděpodobně mu bude lepší doporučit nějaký starší film. Upravíme tedy náš systém tak, aby doporučoval pouze filmy ze stjené dekády.

Nejprve do tabulky `jaccard_distances` vlož sloupec `decade` z tabulky `data`. Pokud si nejsi jistý(á), jak to provést, použij řádek níže.

```py
jaccard_distances["decade"] = data["decade"]
```

Dále se vraťme k filmu The Fugitive. Z lekce víc, z jaké dekády tento film pochází, ale můžeš to zkusit zjistit pomocí `.loc`. Dále vytvoř tabulku `movies_from_decade`, která bude obsahovat pouze filmy ze stejné dekády jako The Fugitive. Nakonec z této tabulky vyber filmy, které jsou svými žánry nejvíce podobné filme The Fugitive. K tomu můžeš použít stejný kód jaký na závěru předchozího příkladu, pouze jej použij s tabulkou `movies_from_decade`.

## Videa

- [Řešení příkladu](https://youtu.be/t0GoyHXxmmk?si=9UilFBIJCN8MeFuh)
- [Řešení bonusu](https://youtu.be/HWBMXsbHyhQ?si=8Au7gizBcP8fS1Ro)
- [Filtrování podle uživatelského profilu](https://youtu.be/vO6hJi0AjeA?si=851MbYP_IaKaT3OV)
- [Kolaborativní filtrování](https://youtu.be/_WGdpnFUEmo?si=ZL7pxstcJnOZFpaf)